# Bibliotecas


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
import logging
import ee
import folium

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

GPU_AFFINTY  = 0
GPU_MEMORY_LIMIT_GB =8 # For UNET use 8GB

logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)
# GPU_AFFINTY  = 0 #GeForce RTX 4090

gpu_dict = {'4090':{'GPU_AFFINTY' : 0, 'GPU_MEMORY_LIMIT_GB':16}, 
            '2070':{'GPU_AFFINTY':1, 'GPU_MEMORY_LIMIT_GB':8}}


sel_gpu = '4090'
GPU_AFFINTY  = gpu_dict[sel_gpu]['GPU_AFFINTY'] #GeForce RTX 2070
GPU_MEMORY_LIMIT_GB =gpu_dict[sel_gpu]['GPU_MEMORY_LIMIT_GB']

try:
 # ee.Authenticate()
 ee.Initialize()
except:
 ee.Authenticate()
 ee.Initialize()

In [ ]:
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Tensorflow Version:',tf.__version__)
print('Folium Version:',folium.__version__)

gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    tf.config.set_visible_devices(gpus[GPU_AFFINTY], 'GPU')
    GPU_MEMORY_LIMIT_GB = GPU_MEMORY_LIMIT_GB * 1e3
    if GPU_MEMORY_LIMIT_GB == 0:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        tf.config.set_logical_device_configuration(gpus[GPU_AFFINTY],[tf.config.LogicalDeviceConfiguration(memory_limit=GPU_MEMORY_LIMIT_GB)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)

In [3]:
def creatDirectory(new_folder):
    if not os.path.exists(new_folder):
        print(f'lets make the directory: {new_folder}')
        os.makedirs(new_folder)
    else: return

# ENV Configs

In [4]:
VERSION        = '2_BR'
MOSAIC_VERSION = '4'
BUCKET = 'mineracao_br'
GDRIVE = 'mb9-unet-mining-brazil'
FOLDER = 'training_samples'
TRAINING_BASE = 'training_patches_'+MOSAIC_VERSION
EVAL_BASE     = 'eval_patches_'+MOSAIC_VERSION

#Local paths
LOCAL_PATH  = '/mnt/storage10.2/mb9-unet-mineracao'
MODEL_DIR   = LOCAL_PATH+'/checkpoint/v'+VERSION
NFS_PATH = '/mnt/nfs/assets/Mapbiomas/modelos/mb9-unet-mineracao'
OUTPUT_PATH = NFS_PATH+'/output/v'+VERSION
GRID_PATH = "/mnt/nfs/assets/Mapbiomas/GRID-ALLCALSSES-COL9.geojson"

creatDirectory(MODEL_DIR)
creatDirectory(OUTPUT_PATH)

# Exportation Configs
BUCKET_patch = BUCKET
FOLDER_patch = 'allPatch'
FOLDER_classification = 'mb9_mining_'+VERSION

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['swir1', 'nir', 'red','green','MNDWI','NDVI'] #['swir1', 'nir', 'red','NDVI','MNDWI'] DEFAULT
BANDS    = opticalBands
RESPONSE = 'supervised'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE  = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 0
EVAL_SIZE = 0

print(BANDS)
# Specify model training parameters.
BATCH_SIZE  = 10
DROPOUT     = 0.3 #0.5
EPOCHS      = 50
BUFFER_SIZE = 1000
OPTIMIZER   = 'Nadam' 
LOSS        = 'BinaryCrossentropy'
METRICS     = ['RootMeanSquaredError']

['swir1', 'nir', 'red', 'green', 'MNDWI', 'NDVI']


# Data Visualization - MG

In [ ]:
mosaic_year   = 2023

# 3_MG E 4_MG
# supervisedImg = ee.Image('projects/solved-mb9/assets/Mining/supervised-2023-2-MG').gte(1).unmask(0).rename(RESPONSE);

# 5_MG: 2023 plus 1989 samples
supervisedImg = ee.Image('projects/solved-mb9/assets/Mining/supervised-2023-6-MG').gte(1).unmask(0).rename(RESPONSE);
#supervisedImg = ee.Image('projects/solved-mb9/assets/Mining/supervised-1990-4-MG').gte(1).unmask(0).rename(RESPONSE);

supervisedChannel = supervisedImg.toByte().rename(RESPONSE);

image = ee.Image('projects/mapbiomas-workspace/TRANSVERSAIS/ZONACOSTEIRA6/mosaic_'+str(mosaic_year)).addBands(supervisedChannel)
mapid = image.getMapId({'bands': ['swir1', 'nir', 'red'], 'min': 30, 'max': 150})
map = folium.Map(location=[-5.9442, -56.5265])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Planet',
    overlay=True,
    name='Mosaic composite',
  ).add_to(map)
mapid = supervisedChannel.select(RESPONSE).getMapId({'min': 0, 'max': 1, 'pallete':'#ff0000'})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='supervisedLayer',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
featureStack = ee.Image.cat([
  image.select(BANDS).unmask(0),
  image.select(RESPONSE).unmask(0)
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

In [ ]:
# trainingPolys = ee.FeatureCollection('projects/solvedltda/assets/MB7_mining/trainPoly_unet_mb7_5')
# evalPolys = ee.FeatureCollection('projects/solvedltda/assets/MB7_mining/testPoly_unet_mb7_5')


# FOR MG MB9
# trainingPolys3 = ee.FeatureCollection('projects/solved-mb9/assets/Mining/train_mg_v2')
# trainingPolys4 = ee.FeatureCollection('projects/solved-mb9/assets/Mining/train_mg_v3')
# trainingPolys = trainingPolys3.merge(trainingPolys4)
trainingPolys = ee.FeatureCollection('projects/solved-mb9/assets/Mining/train_mg_v4')
evalPolys = ee.FeatureCollection('projects/solved-mb9/assets/Mining/test_mg_v4')
print(trainingPolys.size().getInfo())
print(evalPolys.size().getInfo())

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[-1.3621, -45.2738], zoom_start=5)


folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='training polygons',
  ).add_to(map)
mapid = supervisedChannel.select(RESPONSE).getMapId({'min': 0, 'max': 1, 'pallete':'#ff0000'})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='supervisedLayer',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
TRAINING_BASE= 'training_patches_1_6_MG_2023_FOR_MB9'
# TRAINING_BASE
# EVAL_BASE
EVAL_BASE = 'eval_patches_1_6_MG_2023_FOR_MB9'
FOLDER_TRAIN = 'training_samples_FOR_MB9_MG_2023'
FOLDER_EVAL = 'eval_samples_FOR_MB9_BR_2023'
GDRIVE

# Train/Test Chips Exportation

In [ ]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())
# These numbers determined experimentally.
n = 20 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.

#Add some generalism
TRAIN_SIZE = trainingPolys.size().getInfo()*N
EVAL_SIZE = evalPolys.size().getInfo()*N
print('TRAIN:'+str(TRAIN_SIZE))
print('EVAL:'+str(EVAL_SIZE))

# Export all the training data (in many pieces), with one task 
# per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    folder = GDRIVE+'/'+FOLDER_TRAIN, 
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    folder = GDRIVE+'/'+FOLDER_EVAL, 
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE],
  )
  task.start()

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  print(FEATURES_DICT)
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

In [ ]:
def get_training_dataset():
    """Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
    # TRAINING_BASE = "training_patches_4"
    
    # Novas amostras na train_2
    # glob = '/mnt/storage4/modelos/mb8-unet-mineracao/train/' + TRAINING_BASE + '*'
    
    # glob = '/mnt/storage4/modelos/mb7-unet-mineracao/train/' + TRAINING_BASE + '*'
    
    # MB9 BRASIL
    # TRAINING_BASE = "training_patches_1_FOR_MB9_BR_2022_FROM_MB8"
    # glob = '/mnt/storage10.2/mb9-unet-mineracao/train/v1_BR/' + TRAINING_BASE + '*'
    # glob = '/mnt/storage10.2/mb9-unet-mineracao/train/test44/' + TRAINING_BASE + '*'
    
    # MB9 MG
    TRAINING_BASE = "training_patches_1"
    # glob = '/mnt/storage10.2/mb9-unet-mineracao/train/v5_MG/' + TRAINING_BASE + '*'
    glob = '/mnt/nfs/assets/Mapbiomas/samples/mb9-unet-mining/v6/train/'+TRAINING_BASE + '*'
    print(glob)
    dataset = get_dataset(glob)
    train_size = dataset.reduce(np.int64(0), lambda x,_ : x + 1).numpy()
    print(train_size)  # v1_BR =>42605 | v1_BR_180_TF1 => 46605
    dataset = dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).repeat().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

training = get_training_dataset()

# print(iter(training.take(1)).next())
# 42605
print(training.take(1))

In [ ]:
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())
# These numbers determined experimentally.
n = 20 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.

#Add some generalism
TRAIN_SIZE = trainingPolys.size().getInfo()*N*2
EVAL_SIZE = evalPolys.size().getInfo()*N*2
print('TRAIN:'+str(TRAIN_SIZE))
print('EVAL:'+str(EVAL_SIZE))

In [ ]:
def get_eval_dataset():
    # EVAL_BASE = 'eval_patches_4'
    # Novas amostras na eval_2
    # glob = '/mnt/storage4/modelos/mb7-unet-mineracao/eval/' + EVAL_BASE + '*'
    
    
    # EVAL_BASE = 'eval_patches_4'
    # glob = '/mnt/storage4/modelos/mb7-unet-mineracao/eval/' + EVAL_BASE + '*'
    
    
    # MB9 BRASIL
    # EVAL_BASE = "eval_patches_1_FOR_MB9_BR_2022_FROM_MB8"
    # glob = '/mnt/storage10.2/mb9-unet-mineracao/eval/v1_BR/' + EVAL_BASE + '*'
    
    # MB9 MG
    EVAL_BASE = 'eval_patches_1'
    glob = '/mnt/nfs/assets/Mapbiomas/samples/mb9-unet-mining/v6/eval/' + EVAL_BASE + '*'
    
    print(glob)
    dataset = get_dataset(glob)
    eval_size = dataset.reduce(np.int64(0), lambda x,_ : x + 1).numpy()
    print(eval_size)  # v2 => 600 | v3_mg => 1400
    # dataset = dataset.batch(10).repeat()
    dataset = dataset.batch(1).repeat()
    return dataset

evaluation = get_eval_dataset()
# print(iter(evaluation.take(1)).next())
# 22420
print(evaluation.take(1))

# UNET

In [5]:

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers


def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256 (shape=[256, 256, len(BANDS)
	encoder0_pool, encoder0 = encoder_block(inputs, 64) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 128) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 256) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 512) # 16
	center = conv_block(encoder3_pool, 1024) # 8 center
	decoder4 = decoder_block(center, encoder3, 512) # 16
	decoder3 = decoder_block(decoder4, encoder2, 256) # 32
	decoder2 = decoder_block(decoder3, encoder1, 128) # 64
	decoder1 = decoder_block(decoder2, encoder0, 64) # 128
	dropout = layers.Dropout(DROPOUT, name="dropout", noise_shape=None, seed=None)(decoder1)
	outputs = layers.Conv2D(1, (1, 1),  activation=tf.nn.sigmoid, padding='same', kernel_initializer=tf.keras.initializers.GlorotNormal())(dropout)
	model = models.Model(inputs=[inputs], outputs=[outputs])
	optimizer = tf.keras.optimizers.Adam( 0.000005, name='optimizer')

	model.compile(
		optimizer=optimizer, 
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS]
    )
	return model

# Model Selection/Load

In [6]:
#from tensorflow.python.keras.utils.vis_utils import model_to_dot
#from tensorflow.python.keras.utils import plot_model
from IPython.display import Image
#import pydot
#import graphviz
m = get_model() # UNET
#m = FCN_DK3(2, 256, 256)# FCN-DK
#Image(model_to_dot(m, show_shapes=True, show_layer_names=True, rankdir='TB',expand_nested=False, dpi=60, subgraph=False).create(prog='dot',format='png'))



# =========================== UNET TOTAL BRAZIL (UNRELIABLE) ===========================
# EPOCH = 194#
# CHECK_MODEL_DIR = '/mnt/storage4/modelos/mb7-unet-mineracao/checkpoint/v8/cp-00.ckpt/cp-0100.ckpt/cp-0150.ckpt/cp-0'+str(EPOCH)+'.ckpt'
# #CHECK_MODEL_DIR = MODEL_DIR+'/cp-0'+str(EPOCH)+'.ckpt' 
# #print(CHECK_MODEL_DIR)
# m.load_weights(CHECK_MODEL_DIR)
# print(m.summary())


# =========================== UNET MG MB9 ===========================
#EPOCH = 60# FEITO
#EPOCH = 30# FEITO
#EPOCH = 20# FEITO
# EPOCH = 98# FEITO
EPOCH = 44# FEITO
# EPOCH = 54# 
# CHECK_MODEL_DIR_MG = '/mnt/storage10.2/mb9-unet-mineracao/checkpoint/v7_MG/cp-00'+str(EPOCH)+'.ckpt'

# # CHECK_MODEL_DIR = MODEL_DIR+'/cp-0'+str(EPOCH)+'.ckpt' 
# # creatDirectory(CHECK_MODEL_DIR_MG)
# m.load_weights(CHECK_MODEL_DIR_MG)
# print(m.summary())


# =========================== UNET TOTAL BRAZIL MB9 ===========================
# EPOCH = 0
# MODEL_DIR = '/mnt/storage10.2/mb9-unet-mineracao/checkpoint/v1_BR/cp-0'+str(EPOCH)+'.ckpt'
# creatDirectory(MODEL_DIR)
# # m.load_weights(CHECK_MODEL_DIR_MG)
# print(m.summary())


# =========================== UNET TOTAL BRAZIL MB9 ===========================
EPOCH = 170
MODEL_DIR = '/mnt/storage10.2/mb9-unet-mineracao/checkpoint/v2_BR/cp-00.ckpt/cp-0100.ckpt/cp-0150.ckpt/cp-0'+str(EPOCH)+'.ckpt'
# creatDirectory(MODEL_DIR)
m.load_weights(MODEL_DIR)
print(m.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 6)]      0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, None, None, 64)       3520      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, None, None, 64)       256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, None, None, 64)       0         ['batch_normalization[0][0

In [ ]:
import matplotlib.pyplot as plt
import math
import datetime, os

plt.style.use("ggplot")
checkpoint_path = MODEL_DIR+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#trainingPolysList = trainingPolys.toList(trainingPolys.size())
#evalPolysList = evalPolys.toList(evalPolys.size())
n = 20 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.
#TRAIN_SIZE = trainingPolys.size().getInfo()*N
#EVAL_SIZE = evalPolys.size().getInfo()*N

tensorboard = tf.keras.callbacks.TensorBoard(log_dir='output/v'+VERSION+'/log_model',write_images=True)

cp_callback  = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,verbose=1, save_weights_only=True, period=2)
m.save_weights(checkpoint_path.format(epoch=0))


result = m.fit(x=training,
  epochs=100,
  initial_epoch=0, # REMEBER TO CHANGE THIS INITIAL EPOCH PARAM, WHEN OTHER MODEL HAS BEEN LOADED
  steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
  verbose=1,
  shuffle=True,
  validation_data=evaluation,
  validation_steps=19200,#500,
  callbacks = [cp_callback]) #tensorboard


# Export

In [ ]:
def doExport(out_image_base,index_in, kernel_buffer, roi):
  """Run the image export task.  Block until complete.
  """
  index = index_in
  image = ee.Image('projects/mapbiomas-workspace/TRANSVERSAIS/ZONACOSTEIRA6/mosaic_'+str(index))
  out_image_base2 = out_image_base+'_'+str(index_in)

  task = ee.batch.Export.image.toDrive(
    image = image.select(BANDS).toFloat(),
    description = out_image_base+'_'+str(index),
    folder        = outputBucket,
    fileNamePrefix = 'mosaico_mb9_append_landsat/'+str(index)+'/'+out_image_base+'_'+str(index), 
    scale = 30,
    region = roi,
    fileFormat = 'TFRecord',
    formatOptions = { 
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 157286400
    },
    maxPixels=1e13
  )

  task.start()  
 
  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  #while task.active():
  time.sleep(1)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

# Prediction

In [7]:
VERSION     = '2_BR_append_4'
LOCAL_PATH  = '/mnt/storage10.2/mb9-unet-mineracao'
OUTPUT_PATH =  LOCAL_PATH+'/output/v'+VERSION

In [40]:
print(OUTPUT_PATH)
OUTPUT_PATH_NFS = '/mnt/nfs/assets/Mapbiomas/modelos/mb9-unet-mineracao/output/v2_BR_append_4'
print(OUTPUT_PATH_NFS)
# VERSION = '1_BR'
print(VERSION)
# VERSION = '1_BR_apped_2'
OUTPUT_PATH = OUTPUT_PATH_NFS
# EPOCH = 200
print(EPOCH)

/mnt/storage10.2/mb9-unet-mineracao/output/v2_BR_append_4
/mnt/nfs/assets/Mapbiomas/modelos/mb9-unet-mineracao/output/v2_BR_append_4
2_BR_append_4
170


In [56]:
from PIL import Image
import glob
import rasterio
from rasterio.plot import show
import numpy
import matplotlib.pyplot as plt
from osgeo import ogr
from osgeo import gdal
import json
def doPrediction(out_image_base,index_in,region_id, user_folder, kernel_buffer, region,epochs):
  """Perform inference on exported imagery, upload to Earth Engine.
  """
  out_image_base = out_image_base+'_'+str(index_in)
  print(out_image_base)
  filesList = glob.glob("/mnt/storage14/mosaics/"+str(index_in)+"/"+out_image_base+"*")
  rasterFolder = f'{OUTPUT_PATH}/classifications_tiff/{index_in}'
  rasterURILZW = f'{rasterFolder}/outimage_{VERSION}_{region_id}_{index_in}_{epochs}epochs_lzw.tif'
  

  if os.path.exists(rasterURILZW):
        print('Arquivo já predito \n\n')
        return None
    
  if(len(filesList) == 0):
    print('Sem arquivos')
    return None
  exportFilesList = [s for s in filesList if out_image_base in s]
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f
  
  imageFilesList.sort()
  out_image_asset = user_folder + '/' + out_image_base

  try: 
     jsonText = open(jsonFile)
     mixer = json.load(jsonText)
  except:
     print(f'Grid com problema:{region_id} para o ano:{index_in} \n\n')
     return None  
  
  
  patches = mixer['totalPatches']
  cols = int(mixer["patchesPerRow"])
  rows = int(mixer["totalPatches"]/cols)
  
  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in BANDS
  ]
    
  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)


  def to_tuple(inputs):
    """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Args:
    inputs: A dictionary of tensors, keyed by feature name.
    Returns: 
    A dtuple of (inputs, outputs).
    """
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    # Convert from CHW to HWC
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

  def toTupleImage(dict):
    inputsList = [dict.get(key) for key in BANDS] #BANDS
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked
  
  # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=4)
  imageDataset = imageDataset.map(to_tuple).batch(1)
  # Perform inference.
  #print('Running predictions...')
  #with tf.device('/device:GPU:0'):
  try: 
     predictions = m.predict(imageDataset, steps=patches, verbose=2)
  except:
     print(f'Grid com problema:{region_id} para o ano:{index_in} \n\n')
     return None
    
  patchesPerRow  = mixer['patchesPerRow']
  TotalPatches   = mixer['totalPatches']
  patchDimension = mixer['patchDimensions']

  counter         = 1
  rowCounter      = 1
  globalCounter   = 0
  finalArray      = numpy.array([])
  
  rowArray        = numpy.array([])
  rasterFolder    = OUTPUT_PATH+'/classifications_tiff/'+str(index_in)
  out_image_asset = user_folder + '_' + out_image_base
  creatDirectory(rasterFolder) 
  
  if True:
      for raw_record in predictions:
          raw_record = numpy.squeeze(raw_record)
          rows,cols = raw_record.shape
          raw_record = raw_record[128:384,128:384]
          if rowCounter == 1:
              finalArray = rowArray
          if counter <= patchesPerRow:
              if counter == 1:
                  rowArray = raw_record
              else:
                  rowArray = numpy.concatenate((rowArray,raw_record), axis = 1)
              counter = counter+1
          else:
              counter = 2
              rowCounter = rowCounter+1
              if numpy.array_equal(finalArray,rowArray):
                  finalArray = rowArray
              else:
                  finalArray = numpy.concatenate((finalArray,rowArray),axis=0)
              rowArray = raw_record 
          globalCounter = globalCounter+1
      try: 
         finalArray = numpy.concatenate((finalArray,rowArray),axis=0)
      except:
         print(f'Grid com problema:{region_id} para o ano:{index_in} \n\n')
         return None

      rows,cols = finalArray.shape
      driver    = gdal.GetDriverByName("GTiff")

      finalArray2  = numpy.array([finalArray])
      rasterURI    = rasterFolder+'/UNET_v'+VERSION+'_grid_'+str(region_id)+'_year_'+str(index_in)+'.tif'
      rasterURILZW = rasterFolder + '/outimage_'+VERSION+'_'+str(region_id)+'_'+str(index_in)+'_'+str(epochs)+'epochs_lzw.tif'
        
      with rasterio.open(rasterURI,'w',
              driver="GTiff",
              height=rows,
              width=cols,
              count=1,
              dtype="float32",
              crs=mixer["projection"]["crs"],
              transform=mixer["projection"]["affine"]["doubleMatrix"],
              nodata="nan") as dataset:
                  dataset.write(finalArray2)
      !gdal_translate -of GTiff -co "COMPRESS=LZW" -co "PREDICTOR=2" -co "TILED=YES" {rasterURI} {rasterURILZW}
      !rm {rasterURI}

        
  if True:      
      out_image_file = rasterFolder+'/' + out_image_base + '_'+str(epochs)+'epochs.TFRecord'
      out_image_mixer = rasterFolder+'/'+ out_image_base + '_'+str(epochs)+'epochs.json'
      #----------------------------------TFRECORD WRITE ----------------------------
      writer = tf.io.TFRecordWriter(out_image_file)
      patches = 0
      for predictionPatch in predictions:
        #print('Writing patch ' + str(patches) + '...')
        predictionPatch = predictionPatch[
            x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

        # Create an example.
        example = tf.train.Example(
          features=tf.train.Features(
            feature={
              'classification': tf.train.Feature(
                  float_list=tf.train.FloatList(
                      value=predictionPatch.flatten()))
            }
          )
        )
        # Write the example.
        writer.write(example.SerializeToString())
        patches += 1

      writer.close()
      !cp {jsonFile} {out_image_mixer}

In [21]:
# !pip install pygeoj
import pygeoj
kernel_buffer = [256, 256]
image_base_name = 'allPatch_UNET_grid_'
user_folder = 'projects/samm/Mapbiomas8/'
grid = pygeoj.load(GRID_PATH)

print("Total Features on GRID")
original = [int(geo.properties['id']) for geo in grid]
original.sort()

Total Features on GRID
1311


In [ ]:
original = [int(geo.properties['id'])  for geo in grid if geo.properties['mining']==1]
print(len(original))

original.sort()
print(original)

newlist = [] # empty list to hold unique elements from the list
duplist = [] # empty list to hold the duplicate elements from the list
for i in original:
    if int(i) not in newlist:
        newlist.append(i)
    else:
        duplist.append(i)
print('===========================\n')
print("List of duplicates", duplist)
print("Unique Item List", len(newlist))
print('\n\n')

In [22]:
inside_mg_ids = [
1103, # Mari Indicou
1434, # Julia Indicou AP
1052, # MARI indicou
1708, # Julia Indicou
1657, # Julia Indicou
  1760,
  1964,
  1761,
  1812,
  2016,
  2118,
  2169,
  2220,
  2221,
  2119,
  2170,
  1711,
  1762,
  1967,
  2120,
  2222,
  2018,
  1763,
  2019,
  2070,
  2121,
  1968,
  2223,
  1866,
  1969,
  1867,
  2071,
  2020,
  1918,
  1765,
  1816,
  2021,
  1766,
  2072,
  1919,
  1970,
  1868,
  1715,
  1716,
  1869,
  1971,
  1767,
  1818,
  1920,
  2022,
  1921,
  1870,
  1819,
  1972,
  1768,
  2023,
  2074,
  2075,
  1922,
  2024,
  1973,
  1820,
  1871,
  1770,
  1974,
  1821,
  1822,
  2025,
  2123, #NOVO
    2073, #nOVO
    2122 #nOVO
]

In [ ]:
# OUTPUT_PATH = 
VERSION     = '7_MG'
#Local paths
LOCAL_PATH  = '/mnt/storage10.2/mb9-unet-mineracao'
OUTPUT_PATH = NFS_PATH + '/output/v' + VERSION
creatDirectory(OUTPUT_PATH)

In [ ]:
from os import listdir
from os.path import isfile, join
import fnmatch
import re
# mypath = '/home/mluize/storage14/mosaics_sentinel2/2016'
mypath = '/mnt/storage10.2/mosaics_landsat_2023'
onlyfiles = [int(number) for file in listdir(mypath) if (isfile(join(mypath, file)) and (fnmatch.fnmatch(file, '*.json'))) \
             for number in re.findall("(\d+)", file[29:33])]

# onlyfiles = [file[29:33] for file in listdir(mypath) if (isfile(join(mypath, file)) and (fnmatch.fnmatch(file, '*.json')))]
onlyfiles = [file for file in listdir(mypath) if (isfile(join(mypath, file)) and (fnmatch.fnmatch(file, '*.json')))]

onlyfiles_with_ids = [re.findall(r'_grid_(\d+)_', file) for file in listdir(mypath) if (isfile(join(mypath, file)) \
                      and file.endswith('.json'))]
onlyfiles_with_ids.sort()
# print(onlyfiles_with_ids)
# print(len(onlyfiles_with_ids))


ids_list = [int(id) for sublist in onlyfiles_with_ids for id in sublist]
original = [int(geo.properties['id']) for geo in grid]
original.sort()
ids_list.sort()
print(original,'\n')


newlist = [] # empty list to hold unique elements from the list
duplist = [] # empty list to hold the duplicate elements from the list
# newlist = ids_list
for i in original:
    if int(i) not in newlist:
        newlist.append(i)
    else:
        duplist.append(i)
print("List of duplicates", duplist)
# print("Unique Item List", newlist)
print('\n\n')


In [ ]:
# Run the export.
for region in grid:
    region_id = int(region.properties['id'])
    if int(region_id) in [1103,1434] :
        print('Region:',int(region_id))
      #print(region.geometry.coordinates)
        for y in range(1985,2024): #cesar 1985 a 1987
            doExport(image_base_name+str(region_id)+str('_' + MOSAIC_VERSION),y, kernel_buffer, region.geometry.coordinates[0])

In [ ]:
#from keras import backend as K
import time
image_base_name           = 'allPatch_UNET_grid_'
image_base_name_for_2023  = 'mosaico_mb9_landsat_2023_allPatch_UNET_grid_'
MOSAIC_VERSION_after_2020 = '1' 
MOSAIC_VERSION            = '4' 

for y in range(2022, 2023):
    # if y == 2023: 
    #     image_base_name = image_base_name_for_2023
    # if y > 2020:
    #     MOSAIC_VERSION = MOSAIC_VERSION_after_2020
    start = time.time()
    # print('starting...')
    # !echo 'year={y}' >> log.log
    index = 1
    processed_grids = []
    for region in grid:
        region_id = region.properties['id']
        region_id = int(region_id)
        # if (int(region_id) not in processed_grids) and region.properties['mining'] == 1:
        if region.properties['mining'] == 1 and region_id not in inside_mg_ids:
            processed_grids.append(region_id)
            print(region_id,y)
            user_folder = 'projects/solvedltda/assets/MB8/Mining/unet_'+VERSION+'_'+str(y)+'_lzw'
            doPrediction(image_base_name+str(region_id)+str('_' + MOSAIC_VERSION),y,region_id, user_folder, kernel_buffer, region.geometry.coordinates,EPOCH)
            index = index+1
    end = time.time()
    print('Prediction Time per year = '+str(end - start))
print('DONE')